## Periodic pattern mining on canadian TV logs
<img src="skmine_series.png" alt="logo" style="width: 60%;"/>

### The problem, informally
Let's take a simple example. 

Imagine you set an alarm to wake up every day around 7:30AM, and go to work. Sometimes you wake up a bit earlier (your body anticipates on the alarm), and sometimes a bit later, for example if you press the "snooze" button and refuse to face the fact that you have to wake up.

In python we can load those "wake up" events as logs, and store them in a [pandas.Series](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html), like

In [1]:
import datetime as dt
import pandas as pd
one_day = 60 * 24  # a day in minutes
minutes = [0, one_day - 1, one_day * 2 - 1, one_day * 3, one_day * 4 + 2, one_day * 7]

S = pd.Series("wake up", index=minutes)
start = dt.datetime.strptime("16/04/2020 07:30", "%d/%m/%Y %H:%M")
S.index = S.index.map(lambda e: start + dt.timedelta(minutes=e))
S.index = S.index.round("min")  # minutes as the lowest unit of difference
S

2020-04-16 07:30:00    wake up
2020-04-17 07:29:00    wake up
2020-04-18 07:29:00    wake up
2020-04-19 07:30:00    wake up
2020-04-20 07:32:00    wake up
2020-04-23 07:30:00    wake up
dtype: object

In [2]:
S.index.to_numpy()

array(['2020-04-16T07:30:00.000000000', '2020-04-17T07:29:00.000000000',
       '2020-04-18T07:29:00.000000000', '2020-04-19T07:30:00.000000000',
       '2020-04-20T07:32:00.000000000', '2020-04-23T07:30:00.000000000'],
      dtype='datetime64[ns]')

We can see the wake-up time is not exactly the same every day, but overall a consistent regular pattern seems to emerge.

Now imagine that in addition to wake up times, we also have records of other daily activities (meals, work, household chores, etc.), and that rather than a handful of days, those records span several years and make up several thousands of events

**How would you be able to detect regularities in the data ?**

### Introduction to periodic pattern mining
Periodic pattern mining aims at exploiting regularities not only about `what happens` by finding coordinated event occurrences, but also about `when it happens` and `how it happens`, by **finding consistent inter-occurrence timeintervals**.

Next, we introduce the concept of cycles

#### The cycle : a building block for periodic pattern mining
Here is an explicit example of a cycle

<img src="cycle_color.png" alt="cycle" style="width: 60%;"/>

This definition, while being relatively simple, is general enough to allow us to find regularities in different types of logs

#### Handling noise in our timestamps

Needless to say, it would be too easy if events in our data were equally spaced. As data often comes noisy, we have to be fault tolerant, and allow small errors to sneak into our cycles. 

That's the role of `shift corrections`, which capture the small deviations from perfectly regular periodic repetitions, and allow to reconstruct the (noisy) original sequence of events, using the following relation
<img src="shifts.png" alt="shifts" style="width: 60%;"/>

#### A tiny example with scikit-mine
`scikit-mine` offers a `PeriodicCycleMiner`, out of the box.
You can use it to **detect regularities, in the form of cycles**, in the input data. 

These regularities are submitted to an MDL criterion, so that we do not mistakenly include redundant occurences, nor forget to consider other intervals that would sumarize our data in a better way.

MDL offers a framework to find `the best set of cycles`, i.e the set that gives the most succint representation of the data. And `as humans, we often like to deal with non-redundant, well organized data`.

In [3]:
from skmine.periodic import PeriodicCycleMiner
pcm = PeriodicCycleMiner().fit(S)
pcm.discover()

/home/cregan/fsl/lib/python3.10/site-packages/scikit_mine-0.0.10-py3.10.egg/skmine/periodic/cycles.py:20: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  INDEX_TYPES = (pd.DatetimeIndex, pd.RangeIndex, pd.Int64Index,)


,,start,length,period,cost
wake up,0,2020-04-16 07:30:00,5,1 days 00:00:30,63.130299


You can see one cycle has been extracted for our event `wake up`. The cycle covers the entire business week, but not the last monday separated by the weekend

It has a length of 5 and a period close to 1 day, as expected.

Also, note that we "lost" some information here. Our period of 1 day offers the best summary for this data.
Accessing the little "shifts" as encountered in original data is also possible, with an extra argument in our `.discover` call

In [4]:
pcm.discover()

,,start,length,period,cost
wake up,0,2020-04-16 07:30:00,5,1 days 00:00:30,63.130299


The last column named `dE` contains a list of shifts to apply to our cycle in case we want to reconstruct the original data. Trailing zeros have been removed for efficiency, and their values are `relative to the period`, but we can see there is:
 * a -90 second shift between the 1st and 2nd entry (1day30s - 90s later = waking up at 7:29 on tuesday)
 * a 30 second shift between the 2nd and 3rd entry (1day30s - 30s later = still waking up at 7:29 on wednesday)
 * an 30 second shift between the 3rd and 4th entry (back to 7:30 on thursday)
 * an 90 second shift between the 4th and 5th entry (1day 30s + 90s later = waking up at 7:32 on friday)

Also note that we can get the "uncovered" events, called `redisuals`

In [5]:
pcm.get_residuals()

Series([], dtype: object)

This way `pcm` does not store all the data, but has all information needed to reconstruct it entirely !!

In [6]:
pcm.reconstruct()

2020-04-16 07:30:00    wake up
2020-04-17 07:29:00    wake up
2020-04-18 07:29:00    wake up
2020-04-19 07:30:00    wake up
2020-04-20 07:32:00    wake up
dtype: object

#### **ESTHER CODE INTEGRATION TEST**

In [7]:
from skmine.periodic import PeriodicCycleMiner
import pandas as pd
S_doc = pd.Series("ring_a_bell", [10, 20, 32, 40, 60, 79, 100, 240])
pcm = PeriodicCycleMiner().fit(S_doc)
pcm.discover()

start  length  period       cost
ring_a_bell 1     10       3      11  23.552849
            0     40       4      20  24.665780

In [8]:
from skmine.periodic_esther import PeriodicCycleMiner
import pandas as pd


/home/cregan/fsl/lib/python3.10/site-packages/scikit_mine-0.0.10-py3.10.egg/skmine/periodic_esther/cycles.py:26: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [9]:
pcm = PeriodicCycleMiner().fit(S)


 ---- COLLECTION STATS (Total=1 nb_simple=1)
Code length patterns (1): 67.885186
Code length residuals (1): 15.884194
-- Total code length = 83.769381 (87.895949% of 95.305166)



In [10]:
pcm.discover()

,t0,event,length_major,period_major,cost,type
0,1587022200000000000,(wake up)[r=5 p=8643],5,86430000000000,67.885186,simple


You can see one cycle has been extracted for our event `wake up`. The cycle covers the entire business week, but not the last monday separated by the weekend

It has a length of 5 and a period close to 1 day, as expected.

Also, note that we "lost" some information here. Our period of 1 day offers the best summary for this data.
Accessing the little "shifts" as encountered in original data is also possible, with an extra argument in our `.discover` call

The last column named `dE` contains a list of shifts to apply to our cycle in case we want to reconstruct the original data. Trailing zeros have been removed for efficiency, and their values are `relative to the period`, but we can see there is:
 * a -90 second shift between the 1st and 2nd entry (1day30s - 90s later = waking up at 7:29 on tuesday)
 * a 30 second shift between the 2nd and 3rd entry (1day30s - 30s later = still waking up at 7:29 on wednesday)
 * an 30 second shift between the 3rd and 4th entry (back to 7:30 on thursday)
 * an 90 second shift between the 4th and 5th entry (1day 30s + 90s later = waking up at 7:32 on friday)

Also note that we can get the "uncovered" events, called `redisuals`

In [11]:
pcm.get_residuals()

[(1587627000000000000, 'wake up')]

This way `pcm` does not store all the data, but has all information needed to reconstruct it entirely !!

In [12]:
pcm.reconstruct(0)

[(0, 'wake up'),
 (86430000000000, 'wake up'),
 (172860000000000, 'wake up'),
 (259290000000000, 'wake up'),
 (345720000000000, 'wake up')]

### An example with Canadian TV programs
#### Fetching logs from canadian TV

In this section we are going to load some event logs of TV programs (the `WHAT`), indexed by their broadcast timestamps (the `WHEN`).

`PeriodicCycleMiner` is here to help us discovering regularities (the `HOW`)

In [13]:
from skmine.datasets import fetch_canadian_tv
from skmine.periodic import PeriodicCycleMiner

#### Searching for cycles in TV programs

Remember about the definition of cycles ?
Let's apply it to our TV programs

In our case

* $\alpha$ is the name of a TV program

* $r$ is the number of broadcasts (repetitions) for this TV program (inside this cycle)

* $p$ is the optimal time delta between broadcasts in this cycle. If a program is meant to be live everyday at 14:00PM, then $p$ is likely to be `1 day`

* $\tau$ is the first broadcast time in this cycle

* $dE$ are the shift corrections between the $p$ and the actual broadcast time of an event. If a TV program was scheduled at 8:30:00AM and it went on air at 8:30:23AM the same day, then we keep track of a `23 seconds shift`. This way we can summarize our data (via cycles), and reconstruct it (via shift corrections). 


Finally we are going to dig a little deeper into these cycles, to answer quite complex questions about our logs. We will see that cycles contains usefull information about our input data

In [14]:
ctv_logs = fetch_canadian_tv()
ctv_logs.head()

/home/cregan/fsl/lib/python3.10/site-packages/scikit_mine-0.0.10-py3.10.egg/skmine/datasets/periodic.py:113: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  s = pd.read_csv(


timestamp
2020-08-01 06:00:00            The Moblees
2020-08-01 06:11:00    Big Block Sing Song
2020-08-01 06:13:00    Big Block Sing Song
2020-08-01 06:15:00               CBC Kids
2020-08-01 06:15:00               CBC Kids
Name: canadian_tv, dtype: string

In [15]:
pcm = PeriodicCycleMiner(keep_residuals=True).fit(ctv_logs)
pcm.discover()



# %snakeviz -t pcm.fit(ctv_logs)

/home/cregan/fsl/lib/python3.10/site-packages/scikit_mine-0.0.10-py3.10.egg/skmine/periodic/cycles.py:538: UserWarning: found duplicates in input sequence, removing them
  warnings.warn("found duplicates in input sequence, removing them")
/home/cregan/fsl/lib/python3.10/site-packages/scikit_mine-0.0.10-py3.10.egg/skmine/periodic/cycles.py:403: UserWarning: could not generate candidates for the input sequence,
                the model is left empty
  warnings.warn(
/home/cregan/fsl/lib/python3.10/site-packages/scikit_mine-0.0.10-py3.10.egg/skmine/periodic/cycles.py:403: UserWarning: could not generate candidates for the input sequence,
                the model is left empty
  warnings.warn(
/home/cregan/fsl/lib/python3.10/site-packages/scikit_mine-0.0.10-py3.10.egg/skmine/periodic/cycles.py:403: UserWarning: could not generate candidates for the input sequence,
                the model is left empty
  warnings.warn(
/home/cregan/fsl/lib/python3.10/site-packages/scikit_mine-0.0.10-py3

start  length          period  \
A Kandahar Away                 1 2020-08-02 06:00:00       4 7 days 00:00:00   
                                0 2020-08-03 07:11:00       4 1 days 00:00:00   
Absolutely Toronto              1 2020-08-01 11:00:00       4 7 days 00:00:00   
                                0 2020-08-03 09:48:00       4 1 days 00:00:00   
Across the Line                 0 2020-08-03 07:30:00       4 1 days 00:00:00   
...                                               ...     ...             ...   
Jackie Robinson                 4 2020-08-18 00:30:00       4 0 days 00:30:00   
                                5 2020-08-25 02:00:00       4 0 days 00:30:00   
Jamie & Jimmy's Food Fight Club 0 2020-08-02 07:36:00       5 7 days 00:00:00   
Jamie's 15 Minute Meals         0 2020-08-11 09:12:00       3 6 days 23:59:30   
Jamie's Super Foods             0 2020-08-03 16:00:00       5 1 days 00:00:00   

                                        cost  
A Kandahar Away                 1  49.063694  
                                0  50.757989  
Absolutely Toronto              1  49.260768  
                                0  62.851081  
Across the Line                 0  50.645595  
...                                      ...  
Jackie Robinson                 4  50.805869  
                                5  50.805869  
Jamie & Jimmy's Food Fight Club 0  46.509879  
Jamie's 15 Minute Meals         0  53.459446  
Jamie's Super Foods             0  47.171713  

[131 rows x 4 columns]

`Note` : no need to worry for the warning, it's here to notify duplicate event/timestamp pairs have been found

In [16]:
cycles = pcm.discover()
cycles

start  length          period  \
A Kandahar Away                 1 2020-08-02 06:00:00       4 7 days 00:00:00   
                                0 2020-08-03 07:11:00       4 1 days 00:00:00   
Absolutely Toronto              1 2020-08-01 11:00:00       4 7 days 00:00:00   
                                0 2020-08-03 09:48:00       4 1 days 00:00:00   
Across the Line                 0 2020-08-03 07:30:00       4 1 days 00:00:00   
...                                               ...     ...             ...   
Jackie Robinson                 4 2020-08-18 00:30:00       4 0 days 00:30:00   
                                5 2020-08-25 02:00:00       4 0 days 00:30:00   
Jamie & Jimmy's Food Fight Club 0 2020-08-02 07:36:00       5 7 days 00:00:00   
Jamie's 15 Minute Meals         0 2020-08-11 09:12:00       3 6 days 23:59:30   
Jamie's Super Foods             0 2020-08-03 16:00:00       5 1 days 00:00:00   

                                        cost  
A Kandahar Away                 1  49.063694  
                                0  50.757989  
Absolutely Toronto              1  49.260768  
                                0  62.851081  
Across the Line                 0  50.645595  
...                                      ...  
Jackie Robinson                 4  50.805869  
                                5  50.805869  
Jamie & Jimmy's Food Fight Club 0  46.509879  
Jamie's 15 Minute Meals         0  53.459446  
Jamie's Super Foods             0  47.171713  

[131 rows x 4 columns]

The resulting dataframe has a [MultIndex](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.MultiIndex.html).

The first level is the event name, the second level corresponds to the cycle number, as we can detect multiple cycles for the same event

Now that we have our cycles in a [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html), we can play with the pandas API and answer questions about our logs

#### Did I find cycles for the TV show "Arthurt Shorts"

In [17]:
cycles.loc["Arthur Shorts"]

,start,length,period,cost
0,2020-08-03 23:00:00,4,1 days,50.297673


#### What are the top 10 longest cycles ?

In [18]:
cycles.nlargest(10, ["length"])

,,start,length,period,cost
Dr. Seuss' The Lorax,0,2020-08-06 00:30:00,7,0 days 00:30:00,54.987582
Big Block Sing Song,0,2020-08-02 08:57:00,5,7 days 00:00:00,34.509879
Bondi Vet,0,2020-08-01 07:40:00,5,7 days 00:00:00,46.509879
CBC Arts: Exhibitionists,0,2020-08-02 07:30:00,5,7 days 00:00:00,34.509879
CBC Winnipeg Comedy Festival,0,2020-08-02 07:34:00,5,7 days 00:00:00,46.509879
Ethiopian Musicians,0,2020-08-10 08:59:00,5,1 days 00:00:00,57.686133
In the Making,0,2020-08-01 11:48:00,5,7 days 00:00:00,40.509879
Interrupt This Program,0,2020-08-02 11:00:00,5,7 days 00:00:00,34.509879
Jamie & Jimmy's Food Fight Club,0,2020-08-02 07:36:00,5,7 days 00:00:00,46.509879
Jamie's Super Foods,0,2020-08-03 16:00:00,5,1 days 00:00:00,47.171713


#### what are the 10 most unpunctual TV programs ?
For this we are going to :
 1. extract the shift corrections along with other informations about our cycles
 2. compute the sum of the absolute values for the shift corrections, for every cycles
 3. get the 10 biggest sums

In [19]:
full_cycles = pcm.discover(shifts=True)
full_cycles.head()

start  length period       cost  \
A Kandahar Away    1 2020-08-02 06:00:00       4 7 days  49.063694   
                   0 2020-08-03 07:11:00       4 1 days  50.757989   
Absolutely Toronto 1 2020-08-01 11:00:00       4 7 days  49.260768   
                   0 2020-08-03 09:48:00       4 1 days  62.851081   
Across the Line    0 2020-08-03 07:30:00       4 1 days  50.645595   

                                                  dE  
A Kandahar Away    1                       [0, 0, 0]  
                   0                       [0, 0, 0]  
Absolutely Toronto 1                       [0, 0, 0]  
                   0  [0, -60000000000, 60000000000]  
Across the Line    0                       [0, 0, 0]

In [20]:
def absolute_sum(*args):
    return sum(map(abs, *args))

# level 0 is the name of the TV program
shift_sums = full_cycles["dE"].map(absolute_sum).groupby(level=[0]).sum()
shift_sums.nlargest(10)

Frankie Drake Mysteries         1200000000000
CBC News: The National           300000000000
Anne with an E                   240000000000
Daniel Tiger's Neighbourhood     240000000000
Grand Designs                    240000000000
Burden of Truth                  180000000000
Ethiopian Musicians              180000000000
Absolutely Toronto               120000000000
Bondi Vet                        120000000000
CBC Winnipeg Comedy Festival     120000000000
Name: dE, dtype: int64

#### What TV programs have been broadcasted every day for at least 5 days straight?
Let's make use of the [pandas.DataFrame.query](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.query.html) method to express our question in an SQL-like syntax

In [21]:
cycles.query('length >= 5 and period >= "1 days"', engine='python')

,,start,length,period,cost
Big Block Sing Song,0,2020-08-02 08:57:00,5,7 days,34.509879
Bondi Vet,0,2020-08-01 07:40:00,5,7 days,46.509879
CBC Arts: Exhibitionists,0,2020-08-02 07:30:00,5,7 days,34.509879
CBC Winnipeg Comedy Festival,0,2020-08-02 07:34:00,5,7 days,46.509879
Ethiopian Musicians,0,2020-08-10 08:59:00,5,1 days,57.686133
In the Making,0,2020-08-01 11:48:00,5,7 days,40.509879
Interrupt This Program,0,2020-08-02 11:00:00,5,7 days,34.509879
Jamie & Jimmy's Food Fight Club,0,2020-08-02 07:36:00,5,7 days,46.509879
Jamie's Super Foods,0,2020-08-03 16:00:00,5,1 days,47.171713


#### What TV programs are broadcast only on business days ?
From the previous query we see we have a lot of 5-length cycles, with periods of 1 day.
An intuition is that these cycles take place on business days. Let's confirm this by considering cycles with
 1. start timestamps on mondays
 2. periods of roughly 1 day  

In [22]:
monday_starts = cycles[cycles.start.dt.weekday == 0]  # start on monday
monday_starts.query('length == 5 and period >= "1 days"', engine='python')

,,start,length,period,cost
Ethiopian Musicians,0,2020-08-10 08:59:00,5,1 days,57.686133
Jamie's Super Foods,0,2020-08-03 16:00:00,5,1 days,47.171713


### **ESTHER CODE INTEGRATION TEST** with Canadian TV programs
#### Fetching logs from canadian TV

In this section we are going to load some event logs of TV programs (the `WHAT`), indexed by their broadcast timestamps (the `WHEN`).

`PeriodicCycleMiner` is here to help us discovering regularities (the `HOW`)

In [23]:
from skmine.datasets import fetch_canadian_tv
from skmine.periodic_esther import PeriodicCycleMiner

#### Searching for cycles in TV programs

Remember about the definition of cycles ?
Let's apply it to our TV programs

In our case

* $\alpha$ is the name of a TV program

* $r$ is the number of broadcasts (repetitions) for this TV program (inside this cycle)

* $p$ is the optimal time delta between broadcasts in this cycle. If a program is meant to be live everyday at 14:00PM, then $p$ is likely to be `1 day`

* $\tau$ is the first broadcast time in this cycle

* $dE$ are the shift corrections between the $p$ and the actual broadcast time of an event. If a TV program was scheduled at 8:30:00AM and it went on air at 8:30:23AM the same day, then we keep track of a `23 seconds shift`. This way we can summarize our data (via cycles), and reconstruct it (via shift corrections). 


Finally we are going to dig a little deeper into these cycles, to answer quite complex questions about our logs. We will see that cycles contains usefull information about our input data

In [24]:
ctv_logs = fetch_canadian_tv()
ctv_logs.head()

/home/cregan/fsl/lib/python3.10/site-packages/scikit_mine-0.0.10-py3.10.egg/skmine/datasets/periodic.py:117: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  s = pd.read_csv(p, **kwargs)


timestamp
2020-08-01 06:00:00            The Moblees
2020-08-01 06:11:00    Big Block Sing Song
2020-08-01 06:13:00    Big Block Sing Song
2020-08-01 06:15:00               CBC Kids
2020-08-01 06:15:00               CBC Kids
Name: canadian_tv, dtype: string

Compute only simple cycles on ctv_logs with the karg *complex=False* : 

In [25]:
pcm = PeriodicCycleMiner().fit(ctv_logs, complex=False)
pcm.discover()

/home/cregan/fsl/lib/python3.10/site-packages/scikit_mine-0.0.10-py3.10.egg/skmine/periodic_esther/cycles.py:139: UserWarning: found duplicates in input sequence, removing them
  warnings.warn("found duplicates in input sequence, removing them")


 ---- COLLECTION STATS (Total=231 nb_simple=231)
Code length patterns (231): 13607.850841
Code length residuals (507): 11994.387863
-- Total code length = 25602.238704 (71.289653% of 35912.979983)



,t0,event,length_major,period_major,cost,type
0,1596258000000000000,(Grand Designs)[r=31 p=8640],31,86400000000000,101.455576,simple
1,1596267780000000000,(CBC Kids)[r=26 p=8640],26,86400000000000,154.406032,simple
2,1598572800000000000,(Schitt's Creek)[r=8 p=180],8,1800000000000,62.620856,simple
3,1596265140000000000,(CBC Kids)[r=11 p=25920],11,259200000000000,93.039502,simple
4,1596587400000000000,(Kim's Convenience)[r=7 p=180],7,1800000000000,60.844246,simple
...,...,...,...,...,...,...
226,1597154400000000000,(Jamie & Jimmy's Food Fight Club)[r=4 p=8640],4,86400000000000,55.702844,simple
227,1596281100000000000,(CBC Kids)[r=4 p=86004],4,860040000000000,56.979765,simple
228,1597658460000000000,(CBC Kids)[r=5 p=8640],5,86400000000000,57.235173,simple
229,1596439380000000000,(CBC Kids)[r=5 p=8640],5,86400000000000,57.235173,simple


Compute simple and complex (with horizontal and vertical combinations) cycles on ctv_logs : 

In [26]:
pcm = PeriodicCycleMiner().fit(ctv_logs)

# %snakeviz -t pcm.fit(ctv_logs)

/home/cregan/fsl/lib/python3.10/site-packages/scikit_mine-0.0.10-py3.10.egg/skmine/periodic_esther/cycles.py:139: UserWarning: found duplicates in input sequence, removing them
  warnings.warn("found duplicates in input sequence, removing them")


 ---- COLLECTION STATS (Total=115 nb_simple=61 nb_concat=47 nb_other=6 nb_nested=1)
Code length patterns (115): 10880.109756
Code length residuals (454): 10805.204927
-- Total code length = 21685.314682 (60.382944% of 35912.979983)



In [27]:
cycles = pcm.discover()

`Note` : no need to worry for the warning, it's here to notify duplicate event/timestamp pairs have been found

Now that we have our cycles in a [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html), we can play with the pandas API and answer questions about our logs

#### Did I find cycles for the TV show "Arthurt Shorts"

In [28]:
cycles

,t0,event,length_major,period_major,cost,type
0,1596258000000000000,(Grand Designs)[r=31 p=8640],31,86400000000000,101.455576,simple
1,1596427200000000000,((CBC News: The National [d=2880] CBC News Net...,4,604800000000000,472.886395,other
2,1596501000000000000,((This Hour Has 22 Minutes)[r=7 p=180] [d=5382...,3,604800000000000,255.643584,other
3,1596524400000000000,((CBC Kids [d=258] CBC Kids)[r=4 p=60480])[r=4...,4,86400000000000,178.142750,other
4,1596269340000000000,(CBC Kids [d=300] CBC Kids [d=6] Kingdom Force...,5,604800000000000,188.381406,concat
...,...,...,...,...,...,...
110,1596531480000000000,(CBC Kids)[r=4 p=34272],4,342720000000000,55.149034,simple
111,1597648980000000000,(Big Block Sing Song)[r=5 p=8643],5,86430000000000,75.235039,simple
112,1596617940000000000,(CBC Kids [d=558] Daniel Tiger's Neighbourhood...,3,604800000000000,94.546656,concat
113,1597057140000000000,(CBC Kids [d=2724] News)[r=4 p=60480],4,604800000000000,94.856042,concat


In [29]:
cycles[cycles["event"].apply(lambda x: "Arthur Shorts" in x)]

,t0,event,length_major,period_major,cost,type
7,1596448080000000000,((Arthur Shorts [d=78] CBC Kids)[r=5 p=8640])[...,3,604740000000000,193.816983,other
36,1596279600000000000,(Arthur Shorts)[r=5 p=60480],5,604800000000000,54.030035,simple
81,1598262480000000000,(Arthur Shorts [d=258] CBC Kids)[r=4 p=8640],4,86400000000000,114.108878,concat


#### What are the top 10 longest cycles ?

In [30]:
cycles.nlargest(10, ["length_major"])

,t0,event,length_major,period_major,cost,type
0,1596258000000000000,(Grand Designs)[r=31 p=8640],31,86400000000000,101.455576,simple
68,1597648920000000000,(CBC Kids)[r=12 p=8640],12,86400000000000,111.340664,simple
10,1598572800000000000,(Schitt's Creek)[r=8 p=180],8,1800000000000,62.620856,simple
16,1596587400000000000,(Kim's Convenience)[r=7 p=180],7,1800000000000,60.844246,simple
17,1598401800000000000,(Kim's Convenience)[r=7 p=180],7,1800000000000,60.844246,simple
18,1596673800000000000,(Mr. D)[r=7 p=180],7,1800000000000,60.844246,simple
19,1596760200000000000,(Schitt's Creek)[r=7 p=180],7,1800000000000,60.844246,simple
29,1598167620000000000,(CBC Kids)[r=6 p=8640],6,86400000000000,58.858624,simple
50,1597136340000000000,(CBC Kids)[r=6 p=17280],6,172800000000000,58.549426,simple
4,1596269340000000000,(CBC Kids [d=300] CBC Kids [d=6] Kingdom Force...,5,604800000000000,188.381406,concat


In [31]:
pcm.reconstruct(1)

[(0, 'CBC News: The National'),
 (28800000000000, 'CBC News Network'),
 (86400000000000, 'CBC News: The National'),
 (115200000000000, 'CBC News Network'),
 (172800000000000, 'CBC News: The National'),
 (201600000000000, 'CBC News Network'),
 (259200000000000, 'CBC News: The National'),
 (288000000000000, 'CBC News Network'),
 (345600000000000, 'CBC News: The National'),
 (374400000000000, 'CBC News Network'),
 (11460000000000, 'Addison'),
 (12600000000000, 'Beat Bugs'),
 (97860000000000, 'Addison'),
 (99000000000000, 'Beat Bugs'),
 (184260000000000, 'Addison'),
 (185400000000000, 'Beat Bugs'),
 (270660000000000, 'Addison'),
 (271800000000000, 'Beat Bugs'),
 (357060000000000, 'Addison'),
 (358200000000000, 'Beat Bugs'),
 (25140000000000, 'CBC Kids'),
 (25200000000000, "Dragons' Den"),
 (111540000000000, 'CBC Kids'),
 (111600000000000, "Dragons' Den"),
 (197940000000000, 'CBC Kids'),
 (198000000000000, "Dragons' Den"),
 (284340000000000, 'CBC Kids'),
 (284400000000000, "Dragons' Den"),


In [32]:
pcm.get_residuals()

[(1596533700000000000, 'Arthur Shorts'),
 (1596286800000000000, 'Bondi Vet'),
 (1598374800000000000, 'Heartland'),
 (1597892400000000000, 'Mr. D'),
 (1598886000000000000, "Jamie's Super Foods"),
 (1596497400000000000, 'Hello Goodbye'),
 (1597908480000000000, 'CBC Kids'),
 (1597562100000000000, 'PJ Masks'),
 (1598525880000000000, 'CBC Kids'),
 (1597390140000000000, 'CBC Kids'),
 (1598481840000000000, 'CBC News: The National'),
 (1598725800000000000, 'Hockey Central'),
 (1597781640000000000, "Kim's Convenience"),
 (1597806000000000000, "Kim's Convenience"),
 (1597306080000000000, 'PJ Masks'),
 (1596974400000000000, 'Basketball'),
 (1596915000000000000, 'OLYMPIC GAMES REPLAY'),
 (1596891600000000000, 'CEBL Championship 2019.'),
 (1598646600000000000, "You Can't Ask That"),
 (1597006800000000000, 'CBC Winnipeg Comedy Festival'),
 (1598810400000000000, 'When Calls the Heart'),
 (1598305380000000000, 'Just For Laughs: Gags'),
 (1598859840000000000, 'Kingdom Force'),
 (1598171580000000000, 'O

#### what are the 10 most unpunctual TV programs ?
For this we are going to :
 1. extract the shift corrections along with other informations about our cycles
 2. compute the sum of the absolute values for the shift corrections, for every cycles
 3. get the 10 biggest sums

In [33]:
# full_cycles = pcm.discover()
# full_cycles.head()

In [34]:
# def absolute_sum(*args):
#     return sum(map(abs, *args))

# # level 0 is the name of the TV program
# shift_sums = full_cycles["dE"].map(absolute_sum).groupby(level=[0]).sum()
# shift_sums.nlargest(10)

<!-- #### what are the 10 most unpunctual TV programs ?
For this we are going to :
 1. extract the shift corrections along with other informations about our cycles
 2. compute the sum of the absolute values for the shift corrections, for every cycles
 3. get the 10 biggest sums -->

In [35]:
# def absolute_sum(*args):
#     return sum(map(abs, *args))

# # level 0 is the name of the TV program
# shift_sums = full_cycles["dE"].map(absolute_sum).groupby(level=[0]).sum()
# shift_sums.nlargest(10)

#### What TV programs have been broadcasted every day for at least 5 days straight?
Let's make use of the [pandas.DataFrame.query](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.query.html) method to express our question in an SQL-like syntax

In [36]:
# cycles.query('length_major >= 5 and period_major >= 3600', engine='python')
# # cycles.query('length >= 5 and period >= "1 days"', engine='python')

#### What TV programs are broadcast only on business days ?
From the previous query we see we have a lot of 5-length cycles, with periods of 1 day.
An intuition is that these cycles take place on business days. Let's confirm this by considering cycles with
 1. start timestamps on mondays
 2. periods of roughly 1 day  

In [37]:
# monday_starts = cycles[cycles.start.dt.weekday == 0]  # start on monday
# monday_starts.query('length == 5 and period >= "1 days"', engine='python')

References
----------

1.
    Galbrun, E & Cellier, P & Tatti, N & Termier, A & Crémilleux, B
    "Mining Periodic Pattern with a MDL Criterion"

2.
    Galbrun, E
    "The Minimum Description Length Principle for Pattern Mining : A survey"

3. 
    Termier, A
    ["Periodic pattern mining"](http://people.irisa.fr/Alexandre.Termier/dmv/DMV_Periodic_patterns.pdf) 